In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pprint
import geopy
from geopy.distance import great_circle
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [46]:
df = pd.read_excel("benxemydinh_final_ver.xlsx")
df.head()

# df.join(
#     df['Journeys'].str.split( ' –  ', 1, expand=True).rename(columns={0:'Start_point', 1:'Destination'})
# )
# df['Destination'] = new_col[1]

df[['Start_point', "Destination"]] = df.Journeys.str.split(" – ", expand=True)
df.info()
df.to_excel("benxemydinh_final_ver.xlsx")
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company        201 non-null    object
 1   Journeys       201 non-null    object
 2   Times_per_day  201 non-null    int64 
 3   Times_start    201 non-null    object
 4   Times_end      201 non-null    object
 5   Total_time     201 non-null    object
 6   Payment        201 non-null    int64 
 7   Start_point    201 non-null    object
 8   Destination    201 non-null    object
dtypes: int64(2), object(7)
memory usage: 14.3+ KB


,Company,Journeys,Times_per_day,Times_start,Times_end,Total_time,Payment,Start_point,Destination
0,XE KHÁCH BẢO YẾN,Hà Nội – Tuyên Quang,5,17:30,20:00,2 giờ 30 phút,100000,Hà Nội,Tuyên Quang
1,XE KHÁCH BẢO YẾN,Hà Nội – Tuyên Quang,2,17:00,19:30,4 giờ,160000,Hà Nội,Tuyên Quang
2,XE KHÁCH HẢI NAM,Hà Nội – Sơn La,6,16:00,18:30,8 giờ,180000,Hà Nội,Sơn La
3,XE KHÁCH ĐẠI PHÁT,Hà Nội – Lào Cai,1,14:00,16:30,9 giờ,250000,Hà Nội,Lào Cai
4,XE KHÁCH THANH LY,Hà Nội – Cao Bằng,3,12:05,14:35,9 giờ,200000,Hà Nội,Cao Bằng
...,...,...,...,...,...,...,...,...,...
196,XE KHÁCH AN BÌNH TÂM,Hà Nội – Nghệ An,2,8:00,11:00,6 giờ 21 phút,200000,Hà Nội,Nghệ An
197,XE KHÁCH ẤT HÀ,Hà Nội – Nghệ An,1,8:30,11:30,5 giờ 30 phút,170000,Hà Nội,Nghệ An
198,XE KHÁCH ĐỨC LAN,Hà Nội – Nghệ An,7,9:00,12:00,5 giờ,170000,Hà Nội,Nghệ An
199,XE KHÁCH HOÀNG ANH - NGHỆ AN,Hà Nội – Nghệ An,3,9:10,12:10,7 giờ,180000,Hà Nội,Nghệ An


In [47]:
distances_city = pd.read_excel("cities.xlsx")
distances_city.describe(include="all")
distances_city

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Hồ Chí Minh,Ho Chi Minh City,10.8167,106.6333,Vietnam,VN,VNM,Hồ Chí Minh,admin,13312000.0,1704774326
1,Hà Nội,Hanoi,21.0245,105.8412,Vietnam,VN,VNM,Hà Nội,primary,7785000.0,1704413791
2,Hải Phòng,Haiphong,20.8000,106.6667,Vietnam,VN,VNM,Hải Phòng,admin,2103500.0,1704000623
3,Cần Thơ,Can Tho,10.0333,105.7833,Vietnam,VN,VNM,Cần Thơ,admin,1237300.0,1704783472
4,Biên Hòa,Bien Hoa,10.9575,106.8426,Vietnam,VN,VNM,Đồng Nai,admin,1104000.0,1704863046
...,...,...,...,...,...,...,...,...,...,...,...
72,Phú Quốc,Phu Quoc,10.2289,103.9572,Vietnam,VN,VNM,Kiến Giang,minor,70000.0,1704992616
73,Bình Long,Binh Long,11.6504,106.6000,Vietnam,VN,VNM,Bình Phước,NaN,40279.0,1704953411
74,Luân Châu,Luan Chau,21.7400,103.3430,Vietnam,VN,VNM,Điện Biên,NaN,7335.0,1704201464
75,Gia Lai,Gia Lai,13.8079,108.1094,Vietnam,VN,VNM,Gia Lai,admin,NaN,1704264596


In [50]:
# Code comment:
# for col in distance.columns:
#     if col == 'city':
#         start_city = distance[col][1]
#     elif col == 'lat':
#         start_city_lat = distance[col][1]
#     elif col == 'lng':
#         start_city_lng = distance[col][1]

def get_start_city_attr(city_col, lat_col, lng_col):
    start_city = []
    if city_col == 'admin_name' and lat_col == 'lat' and lng_col == 'lng':
        for i in range(75):
            if distances_city[city_col][i] == "Hà Nội":
                start_city.append(distances_city[city_col][i])
                start_city.append(distances_city[lat_col][i])
                start_city.append(distances_city[lng_col][i]) 
    return start_city

def get_destination_city_attr(city_col, lat_col, lng_col):
    destination_cities =[]
    if city_col == 'admin_name' and lat_col == 'lat' and lng_col == 'lng':
        for i in range(75):
            if distances_city[city_col][i] != "Hà Nội":
                destination_cities.append(distances_city[city_col][i])
                destination_cities.append(distances_city[lat_col][i])
                destination_cities.append(distances_city[lng_col][i]) 
    return destination_cities

# Using to generate a sublists from a list, each element in the sublists can not be subscripted:
def chunks(list, n):
    return [list[i: i+n] for i in range(0,  len(list), n)]

# This method have the same executable value like the above method:
# def chunks_2(list, n):
#     for i in range(0, len(list), n):
#         yield list[i: i+n]

start_point = get_start_city_attr('admin_name', 'lat', 'lng')
destination_points = get_destination_city_attr('admin_name', 'lat', 'lng')
destination_point = [destination_points[i: i+3] for i in range(0, len(destination_points), 3)]
des_cities_size = len(destination_point)
print(des_cities_size)

def measure_distance():
    measures =[]
    lat_lng_1 = (start_point[1], start_point[2])
    for i in range(des_cities_size):
        lat_lng_2 = (destination_point[i][1], destination_point[i][2])
        measures.append(start_point[0])
        measures.append(destination_point[i][0])
        measures.append(great_circle(lat_lng_1, lat_lng_2).km)
    return measures

temp_distance = measure_distance()
distance_between_two_cities = [temp_distance[i: i+3] for i in range(0, len(temp_distance), 3)]

# Adding a new column to the dataframe:
df = df.assign(Distances=pd.Series(np.random.randn(201)).values)

def add_column_distances():
    for i in range(des_cities_size):
        for j in range(201):
            if distance_between_two_cities[i][1] == df['Destination'][j]:
                df.__getitem__('Distances').__setitem__(j, distance_between_two_cities[i][2])
    return df

# for i in range(74):
#     for j in range(201):
#         if distance_between_two_cities[i][1] == df['Destination'][j]:
#             values_measure.append(distance_between_two_cities[i][2]) # fix
#             df = df.assign(Distances=pd.Series(values_measure).values) # fix

add_column_distances()
df.info()
df.head()
df.to_excel('benxemydinh_distance.xlsx')
print(distance_between_two_cities)
# pprint.pprint(list(measure_distance()))

# Print the location in lattitude anf longitude of each city:
# print(start_point)
# pprint.pprint(list(chunks(destination_points, 3))) # Generate the chunked lists from destination_points list.
# destination_point[73][2]
# start_point[1]

73
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company        201 non-null    object 
 1   Journeys       201 non-null    object 
 2   Times_per_day  201 non-null    int64  
 3   Times_start    201 non-null    object 
 4   Times_end      201 non-null    object 
 5   Total_time     201 non-null    object 
 6   Payment        201 non-null    int64  
 7   Start_point    201 non-null    object 
 8   Destination    201 non-null    object 
 9   Distances      201 non-null    float64
dtypes: float64(1), int64(2), object(7)
memory usage: 15.8+ KB
[['Hà Nội', 'Hồ Chí Minh', 1138.2025784635184], ['Hà Nội', 'Hải Phòng', 89.30483047552687], ['Hà Nội', 'Cần Thơ', 1222.1830868642112], ['Hà Nội', 'Đồng Nai', 1124.4908156914285], ['Hà Nội', 'Quảng Nam', 607.5910053688576], ['Hà Nội', 'Bắc Ninh', 30.293254207367184], ['Hà Nội', 'Hải Dương', 51.9049406